In [1]:
import pandas as pd
from statsmodels.stats.proportion import power_proportions_2indep, test_proportions_2indep
from statsmodels.stats.weightstats import ttest_ind
from statsmodels.stats.power import tt_ind_solve_power

In [2]:
raw_df = pd.read_csv('./dataset/A-B Testing/ab.csv')
raw_df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [3]:
raw_df['timestamp'] = pd.to_datetime(raw_df['timestamp'])
raw_df['timestamp']

0        2017-01-21 22:11:48.556739
1        2017-01-12 08:01:45.159739
2        2017-01-11 16:55:06.154213
3        2017-01-08 18:28:03.143765
4        2017-01-21 01:52:26.210827
                    ...            
294473   2017-01-03 22:28:38.630509
294474   2017-01-12 00:51:57.078372
294475   2017-01-22 11:45:03.439544
294476   2017-01-15 01:20:28.957438
294477   2017-01-16 12:40:24.467417
Name: timestamp, Length: 294478, dtype: datetime64[ns]

In [4]:
raw_df['date'] = raw_df['timestamp'].dt.date
raw_df['date'] = pd.to_datetime(raw_df['date'])

In [19]:
raw_df_day = raw_df.groupby(['date','group']).agg({'converted':['count','sum']}).reset_index()
raw_df_day.columns = ['date','group','visit','sales']

raw_df_day_visit = pd.pivot_table(index='date',columns=['group'], values=['visit'], data=raw_df_day).reset_index()
raw_df_day_visit.columns = ['date','control_visit','treatment_visit']

raw_df_day_sales = pd.pivot_table(index='date',columns=['group'], values=['sales'], data=raw_df_day).reset_index()
raw_df_day_sales.columns = ['date','control_sales','treatment_sales']

raw_df_day_new = pd.merge(left=raw_df_day_visit, right=raw_df_day_sales).reset_index(drop=True)

raw_df_day_new.head()

,date,control_visit,treatment_visit,control_sales,treatment_sales
0,2017-01-02,2894,2889,362,348
1,2017-01-03,6684,6710,760,762
2,2017-01-04,6655,6629,810,773
3,2017-01-05,6538,6586,801,758
4,2017-01-06,6684,6844,778,848


In [7]:
def get_p_value(raw_df_weekly_new):
    
    result = test_proportions_2indep(count1=raw_df_weekly_new['treatment_sales'],
                                    nobs1=raw_df_weekly_new['treatment_visit'],
                                    count2=raw_df_weekly_new['control_sales'],
                                    nobs2=raw_df_weekly_new['control_visit'],
                                    alternative='two-sided')
    
    return result[1]

def get_power(raw_df_weekly_new):
    
    t_s = raw_df_weekly_new['treatment_visit'] / (raw_df_weekly_new['treatment_visit'] + raw_df_weekly_new['control_visit'])
    r_e = (raw_df_weekly_new['treatment_conversion_rate'] - raw_df_weekly_new['control_conversion_rate']) / raw_df_weekly_new['control_conversion_rate']
    
    power = power_proportions_2indep(diff = raw_df_weekly_new['control_conversion_rate'] * r_e,
                                    prop2 = raw_df_weekly_new['control_conversion_rate'],
                                    nobs1 = raw_df_weekly_new['treatment_visit'],
                                    ratio = (1-t_s)/t_s,
                                    alpha = 0.05,
                                    alternative = 'two-sided',
                                    return_results=False)
    
    return power

In [8]:
raw_df_weekly_new['control_conversion_rate'] = raw_df_weekly_new['control_sales'] / raw_df_weekly_new['control_visit']
raw_df_weekly_new['treatment_conversion_rate'] = raw_df_weekly_new['treatment_sales'] / raw_df_weekly_new['treatment_visit']


raw_df_weekly_new['p_value'] = raw_df_weekly_new.apply(get_p_value, axis=1)
raw_df_weekly_new['power'] = raw_df_weekly_new.apply(get_power, axis=1)

raw_df_weekly_new.head()

,date,control_visit,treatment_visit,control_sales,treatment_sales,control_conversion_rate,treatment_conversion_rate,p_value,power
0,2017-01-02,2894,2889,362,348,0.125086,0.120457,0.592186,0.083522
1,2017-01-03,6684,6710,760,762,0.113704,0.113562,0.979216,0.050078
2,2017-01-04,6655,6629,810,773,0.121713,0.116609,0.364216,0.148248
3,2017-01-05,6538,6586,801,758,0.122515,0.115093,0.189074,0.260414
4,2017-01-06,6684,6844,778,848,0.116397,0.123904,0.179701,0.271593


In [9]:
raw_df_weekly_new.to_csv('./dataset/ready/ab_test_daily_power_pvalue.csv',index=False)

In [14]:
raw_df_group = raw_df.groupby(['group','landing_page']).agg({'converted':['count','sum']}).reset_index()
raw_df_group.columns = ['group','landing_page','visit','sales']
raw_df_group

,group,landing_page,visit,sales
0,control,new_page,1928,234
1,control,old_page,145274,17489
2,treatment,new_page,145311,17264
3,treatment,old_page,1965,250


In [15]:
raw_df_group.to_csv('./dataset/ready/ab_group_page_visit_sales.csv',index=False)

In [20]:
temp_case_1_result = raw_df.groupby(["landing_page"]).agg({'converted':['count','sum']}).reset_index().values
df_case_1_result = pd.DataFrame(temp_case_1_result,columns = ["landing_page","visit","sales"] )
df_case_1_result['conversion_rate'] = df_case_1_result['sales'] / df_case_1_result['visit']
df_case_1_result

,landing_page,visit,sales,conversion_rate
0,new_page,147239,17498,0.118841
1,old_page,147239,17739,0.120478


In [21]:
analysis_case_1_result = test_proportions_2indep(count1=df_case_1_result["sales"][1], 
                                                 nobs1=df_case_1_result["visit"][1], 
                                                 count2=df_case_1_result["sales"][0], 
                                                 nobs2=df_case_1_result["visit"][0], 
                                                 alternative="two-sided")
print(analysis_case_1_result)

statistic = 1.3683036739186678
pvalue = 0.17121704128136572
compare = diff
method = agresti-caffo
diff = 0.0016367945992569882
ratio = 1.0137730026288718
odds_ratio = 1.0156596381009457
variance = 1.4309075737183843e-06
alternative = two-sided
value = 0
tuple = (1.3683036739186678, 0.17121704128136572)
